In [1]:
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# Changed Code for audit integration

cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE)values
(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1), 'RISK_PERSON',GETDATE(),'ERROR','RISK_PERSON','PERSON_MASTER_CLUSTER_FINGATE_REPORT',(SELECT MAX(T.MAX_DATE) FROM (SELECT MAX(CREATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT UNION SELECT MAX(UPDATE_DATE) AS MAX_DATE FROM FINCORE_DB.FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT) T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = 'RISK_PERSON' ",conn)
job_id = job_id.iloc[0,0]
rules_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with(nolock)", conn)
# org_df = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)
conn.commit()
conn.close()


In [4]:
job_id

592

In [5]:
%store job_id

Stored 'job_id' (int64)


In [6]:
rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Individual')]

RULE_ID     RULE_NO                                          RULE_DESC  \
1       1.1  P001001000  Individual exists on FIU or LEA watchlist or S...   
7       1.3  P001003000  Individual is flagged in the international san...   
9       1.4  P001004000                   Adverse Media Screening reported   
11      1.5  P001005000         Multiple PAN/ DIN/ DPIN number(s) reported   
16    2.1.1  P002001001                            Number of bank accounts   
18   2.1.10  P002001010                             Number of credit cards   
20   2.1.11  P002001011               Number of prepaid/wallets/gift cards   
22   2.1.12  P002001012                                Number of addresses   
24   2.1.13  P002001013  Nationality/citizenship country/country of bir...   
27   2.1.15  P002001015                            Number of Directorships   
29   2.1.16  P002001016          Person Related with high risk entity(ies)   
31   2.1.18  P002001018                       Number of brokerage accounts   
33   2.1.19  P002001019                           Number of demat accounts   
35    2.1.2  P002001002                         Accounts in too many banks   
37   2.1.21  P002001021  Number of private banking/wealth management ac...   
39   2.1.23  P002001023                     Number of High Risk STRs filed   
41   2.1.24  P002001024                   Number of Medium Risk STRs filed   
43   2.1.25  P002001025                      Number of Low Risk STRs filed   
45   2.1.26  P002001026                     Number of High Risk CTRs filed   
47   2.1.27  P002001027                   Number of Medium Risk CTRs filed   
49   2.1.28  P002001028                      Number of Low Risk CTRs filed   
51   2.1.29  P002001029                   Number of High Risk CBWTRs filed   
53   2.1.30  P002001030                 Number of Medium Risk CBWTRs filed   
55   2.1.31  P002001031                    Number of Low Risk CBWTRs filed   
57   2.1.32  P002001032                     Number of High Risk NTRs filed   
59   2.1.33  P002001033                   Number of Medium Risk NTRs filed   
61   2.1.34  P002001034                      Number of Low Risk NTRs filed   
63    2.1.4  P002001004           Accounts in high Risk domestic locations   
65    2.1.5  P002001005          Addresses in high Risk domestic locations   
67    2.1.7  P002001007  Number of names/aliases reported across differ...   
69    2.1.8  P002001008                           Number of mobile Numbers   
71    2.1.9  P002001009                                Number of email Ids   
73    2.3.1  P002003001  Total amount of credit is greater than x% of r...   
75    2.3.2  P002003002  Total amount of debit is greater than x% of re...   
77    2.3.3  P002003003  Immovable Property value is greater than x% of...   
79    2.3.5  P002003005  Value of credit card transaction(s) is greater...   
81    2.3.6  P002003006  Value of cash transaction(s) is greater than x...   
83    2.3.7  P002003007  Value of mutual investment(s) is greater than ...   
85    2.3.8  P002003008  Value of investment(s) in capital markets is g...   
89    3.1.1  P003001001                                   Type of customer   
91   3.1.10  P003001010                          Date of last KYC / re-KYC   
93    3.1.2  P003001002                               High Risk profession   
96    3.1.4  P003001004                              Type of accounts held   
99    3.1.7  P003001007  Ageing of Reports Filed (STR, CTR, CBWTR, CCR,...   
101   3.1.8  P003001008                       Individual risk given by RE    
103   3.1.9  P003001009                             Client onboarding date   

    PERSON_TYPE                                          RISK_TYPE  \
1    Individual                                   Auto High Rules    
7    Individual                                   Auto High Rules    
9    Individual                             KYC profile based Risk   
11   Individual                             KYC profile b

In [7]:
#Queries that populate risk_person_summary_stg

master_data_pop_per_q1 = '''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
(
persON_MASTER_ID,CREATE_DATE()
)
SELECT DISTINCT PERSON_MASTER_ID,GETDATE()
FROM FINCORE.PERSON_MASTER_CLUSTER_FINGATE_REPORT PMCFR WITH (NOLOCK)
INNER JOIN FINCORE.PERSON_CLUSTER PC  WITH (NOLOCK)
ON PMCFR.PERSON_CLUSTER_ID = PC.PERSON_CLUSTER_ID
INNER JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.FINGATE_PERSONDETAIL PD WITH (NOLOCK)
ON PC.FINGATE_PERSON_ID = PD.ID_ AND PD.REPORTID = PMCFR.REPORT_ID
where is_active_flag = 1'''




per_pincode_update = '''
UPDATE fincore_db.[Fincore_Analytics].[RISK_person_SUMMARY_STG]
SET PINCODE = B.pin
from 
(select pan as P, pin
from
(
select  pan, try_cast(try_cast(primaryPinCode as int) as decimal(6,0)) as pin,  
rank() over(partition by pan order by legacy_batch_id desc) as orank 
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail] with (nolock)
where pan is not null and primaryPinCode is not null
) T
where orank= 1 

)
B
where pan = B.P'''


per_occ_update = '''UPDATE   [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG] 
   SET [OCCUPATION_BUSINESS] = LOB

from 
(
SELECT PERSON_MASTER_ID,LOB FROM (Select PERSON_MASTER_ID, 
RANK () OVER (PARTITION BY PERSON_MASTER_ID ORDER BY rcd_create_date DESC ) AS RNK, LOB,
RCD_CREATE_DATE 
FROM
(
--CODE FOR FINCORE 2.0
SELECT A.persON_MASTER_ID,
COALESCE( (
SELECT TITLE FROM [DATAHUB_DB].[FIUMetaHub].finnet_OccupationType with (nolock) 
WHERE ID_ = D.occupationId),'Others') as LOB
,d.rcd_create_date 
FROM
fincore_db.Fincore_Analytics.RISK_PERSON_SUMMARY_STG A with (nolock)
inner JOIN fincore_db.Fincore.person_Master_Cluster_Fingate_Report B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID
inner JOIN FINCORE_BRIDGE_DB.FINCORE_BRIDGE.fingate_Persondetail D with (nolock) ON B.Batch_Id = D.batchId AND B.Report_Id = D.reportId
WHERE D.migrated_flag is null  


union
--CODE FOR HDM DATA
sELECT k.PERSON_MASTER_ID,LOB,rcd_create_date FROM 
FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG k with (nolock)
--inner join FINCORE_DB.Fincore.Person_Master PM with (nolock) on k.PERSON_MASTER_ID = PM.Person_Master_Id
INNER JOIN (
select pan ,  upper(LEGACY_OCCUPATION) as LOB,rcd_create_date
from [FINCORE_BRIDGE_DB].[FINCORE_BRIDGE].[fingate_PersonDetail]  with (nolock)
where LEGACY_OCCUPATION is not null  AND migrated_flag = 'y'
group by pan ,  upper(LEGACY_OCCUPATION),rcd_create_date

)A 
ON k.PAN = A.PAN
 

) Z ) Y 
WHERE Y.RNK = 1
) B
where [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG].person_master_id = B.Person_master_id'''




master_data_pop_per_q2 = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]

SET PAN = b.pan , 
DIN = b.din ,
full_name = b.full_name,
Father_Name = b.Father_Name,
Date_Of_Birth = b.Date_Of_Birth,
Ckyc_Number = b.Ckyc_Number,
gender = b.gender

from 

(
select Person_Master_Id as entity_id , 
full_name,
Father_Name , 
Date_Of_Birth ,
gender , 
pan , 
din , 
Ckyc_Number
from FINCORE_DB.Fincore.Person_Master with (nolock)
) b

where person_master_id = b.entity_id'''


master_data_pop_per_income = '''update [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG]
set cbdt_income_range = b.income_range
from (
select pm.Person_Master_Id as entity_id,
case when itr.Gross_Total_Income_Range = 0 then null
when itr.Gross_Total_Income_Range = 1 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 2 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 3 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 4 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 5 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 6 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 7 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 8 then (cbdtir.Max_Value_Lakh + cbdtir.Min_Value_Lakh)/2
when itr.Gross_Total_Income_Range = 9 then 10000000 end as Income_range 
from FINCORE_DB.FINCORE.Person_Master pm inner join EXTL_DB.CBDT.CBDT_Itr_Data itr
on itr.Pan =pm.pan inner join   EXTL_DB.CBDT.CBDT_Income_Range   cbdtir
on itr.Gross_Total_Income_Range=cbdtir.Income_Range 
group by pm.Person_Master_Id,cbdtir.Max_Value_Lakh,cbdtir.Min_Value_Lakh,Gross_Total_Income_Range
)b

where person_master_id = b.entity_id'''



In [8]:
per_trx_update_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_CREDIT = round(average,2)
from (


select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
 
 
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='deposit'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select beneficiaryAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t
on t.beneficiaryAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
and transactionType ='credit'
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id




union 


select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select beneficiaryAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY beneficiaryAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
beneficiaryAcNo is not null and beneficiaryAcNo !=''
group by beneficiaryAcNo ,transactionDate)t 
on t.beneficiaryAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id





)w
group by w.person_master_id
)X

where FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''




per_trx_update_debit = '''
update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TRAN_AMNT_DEBIT = round(average,2)
from (


select w.Person_Master_Id as Person_Master_Id,sum(w.monthlyaverage)as average
from 
(select PMAM.Person_Master_Id ,
 
 
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
avg(amount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
--sum(amount)/Datediff(Month,transactionDate,getdate())  as monthlyaverage
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_Tctwo with (nolock)
where
accountNumber is not null and accountNumber !=''
and depositeWithdrawl ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select accountNumber ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY accountNumber ORDER BY  transactionDate desc ) Rank
from  FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TcOne with (nolock)
where 
accountNumber is not null and accountNumber !=''
and depositWithdrawal ='Withdrawal'
group by accountNumber,transactionDate)t on t.accountNumber=Am.Account_Number 
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select SENDERACNO ,
AVG(Amount) as average,
RANK() OVER (PARTITION BY SENDERACNO ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsOne with (nolock)
where  
SENDERACNO is not null and SENDERACNO !=''
and transactionType ='DEBIT'
group by SENDERACNO ,transactionDate)t
on t.SENDERACNO=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id
 
 

union
 
 

select PMAM.Person_Master_Id ,
sum(t.average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join 
(select senderAcNo ,
AVG (amount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsTwo with (nolock)
where 
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t
on t.senderAcNo=Am.Account_Number 
where Rank  = 1
group by PMAM.Person_Master_Id
 
 
union
 
 
select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_TsThree with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
and transactionType ='DEBIT'
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id




union 


select PMAM.Person_Master_Id ,
sum(t. average) as monthlyaverage
from fincore_db.Fincore.person_master_account_master  PMAM with (nolock)
inner join fincore_db.Fincore.account_master AM with (nolock) on PMAM.Account_Master_Id = am.Account_Master_Id
inner join (select senderAcNo ,
AVG(transactionAmount) as average,
RANK() OVER (PARTITION BY senderAcNo ORDER BY  transactionDate desc ) Rank
from FINCORE_BRIDGE_DB.[FINCORE_BRIDGE].fingate_GenReporting with (nolock)
where  
senderAcNo is not null and senderAcNo !=''
group by senderAcNo ,transactionDate)t 
on t.senderAcNo=Am.Account_Number 
where Rank = 1
group by PMAM.Person_Master_Id


)w
group by w.person_master_id
)X
where 
FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id = X.Person_Master_Id'''




peer_grouping_columns = '''update fincore_db.fincore_analytics.RISK_PERSON_SUMMARY_STG
set PEER_GROUP_AMNT  = Case 
when (CBDT_INCOME_RANGE IS not null  and CBDT_INCOME_RANGE!=0)  then CBDT_INCOME_RANGE
when (TRAN_AMNT_CREDIT Is not null  and TRAN_AMNT_CREDIT!=0) then TRAN_AMNT_CREDIT
when (TRAN_AMNT_DEBIT IS not null  and TRAN_AMNT_DEBIT!=0) then TRAN_AMNT_DEBIT
else 0
end 
'''





In [9]:
# ------Mobile Count 
MOBILE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set MOBILE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'M'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''





# ------TELEPHONE Count 
TELEPHONE_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set TELEPHONE_COUNT= k.no_of_contacts from 
(
SELECT  distinct  PMCM.Person_Master_Id,  
      Count(Distinct PMCM.Contact_Master_Id) as no_of_contacts    
      FROM [FINCORE_DB].[Fincore].Person_Master_Contact_Master PMCM  with(nolock)
      INNER JOIN [FINCORE_DB].[Fincore].Contact_Master CM      with(nolock)
      ON PMCM.Contact_Master_Id = CM.Contact_Master_Id 
      WHERE PMCM.Is_Active_Flag = 1 AND CM.Contact_Type = 'T'  GROUP BY PMCM.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

# ------Email Count 
EMAIL_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set EMAIL_COUNT= k.no_of_emailid from 
(
SELECT B.PERSON_MASTER_ID,  
  COUNT(DISTINCT A.EMAIL_ID) as no_of_emailid       
  FROM  [FINCORE_DB].[Fincore].PERSON_CLUSTER A     with(nolock)
  INNER JOIN  [FINCORE_DB].[Fincore].PERSON_MASTER_CLUSTER_FINGATE_REPORT B   with(nolock)
  ON A.PERSON_CLUSTER_ID = B.PERSON_CLUSTER_ID       
  WHERE A.EMAIL_ID IS NOT NULL    and A.status_active_flag = 1 and    B.is_active_flag =1  
  GROUP BY B.Person_Master_Id)k
      where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ------WATCHLIST COUNT 
WATCHLIST_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set WATCHLIST_COUNT= k.watchlist_count from 
(
select  pm.Person_Master_Id,     
               count(w.Agency_Name) as watchlist_count         
               from  FINCORE_DB.Fincore.Person_Master PM       with(nolock)   
               inner join [FINCORE_DB].[Fincore].[Watchlist] W      with(nolock)  
               on PM.PERSON_MASTER_ID = W.Entity_Master_Id            
               where w.Entity_Type = 'I'   and PM.Is_Active_Flag =  1   and W.Is_Active_Flag = 1 group by pm.Person_Master_Id )k
                 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----STR_REPORT_COUNT
STR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set STR_REPORT_COUNT = K.STR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS STR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''






# -----CTR_REPORT_COUNT
CTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CTR_REPORT_COUNT = K.CTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 
                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''


# -----NTR_REPORT_COUNT
NTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set NTR_REPORT_COUNT = K.NTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS NTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# -----PTR_REPORT_COUNT
PTR_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set PTR_REPORT_COUNT = K.PTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS PTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'PTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



# ---------------CBWTR_REPORT_COUNT
CBWTR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CBWTR_REPORT_COUNT = K.CBWTR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CBWTR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CBWTR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''






# -----CCR_REPORT_COUNT
CCR_REPORT_COUNT = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set CCR_REPORT_COUNT = K.CCR_COUNT from
(SELECT   PMCFR.Person_Master_Id,      
                       COUNT( 1 ) AS CCR_COUNT      
                       FROM [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR with (nolock) 

                       INNER JOIN datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  

                       WHERE PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CCR'  GROUP BY PMCFR.person_Master_Id )K
                       where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''







# ----str_amount_debit
str_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_debit = k.str_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
(( PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----str_amount_credit

str_amount_credit =  '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set str_amount_credit = k.str_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as str_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with (nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'STR' )and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----ctr_amount_debit
ctr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_debit = k.ctr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#  ----ctr_amount_credit

ctr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ctr_amount_credit = k.ctr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ctr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'CTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



# ----ntr_amount_debit
ntr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_debit = k.ntr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR' )and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----ntr_amount_credit

ntr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ntr_amount_credit = k.ntr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ntr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  B.typeName = 'NTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----cbwtr_amount_debit
cbwtr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_debit = k.cbwtr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----cbwtr_amount_credit

cbwtr_amount_credit = ''' update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set cbwtr_amount_credit = k.cbwtr_amount_credit from

 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as cbwtr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CBWTR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''




# ----ccr_amount_debit
ccr_amount_debit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_debit = k.ccr_amount_debit from


(select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_debit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='withdrawal' or t.CR_DR_DEP_WITH='debit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''



#      ----ccr_amount_credit
ccr_amount_credit = '''update FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG set ccr_amount_credit = k.ccr_amount_credit from
 (select PMCFR.Person_Master_Id,SUM(t.TRAN_AMNT) as ccr_amount_credit from [FINCORE_DB].[Fincore].person_Master_Cluster_Fingate_Report PMCFR  with(nolock) inner join 
 datahub_db.FIUMetaHub.[finnet_ReportType] B with (nolock)  ON PMCFR.Report_Type_Id = B.id_  
 inner join FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG t  with(nolock) on     PMCFR.Batch_Id =t.BATCH_ID and PMCFR.Report_Id=t.REPORT_ID  where 
 ((PMCFR.Is_Active_Flag = 1 and  t.REPORT_TYPE = 'CCR') and (t.CR_DR_DEP_WITH='deposit' or t.CR_DR_DEP_WITH='credit')) group by 
 PMCFR.Person_Master_Id)k
 where k.Person_Master_Id =FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG.Person_Master_Id'''

In [10]:
tran_amnt = '''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG SET TRAN_AMNT_DEBIT = T.TRAN_DEBIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH,SUM(TRAN_AMNT) AS TRAN_DEBIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  
WHERE ENTITY_TYPE = 'I' 
GROUP BY ENTITY_ID,ENTITY_TYPE,CR_DR_DEP_WITH
HAVING CR_DR_DEP_WITH = 'DEBIT'
)T
WHERE ENTITY_ID = PERSON_MASTER_ID


UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG SET TRAN_AMNT_CREDIT = T.TRAN_CREDIT
FROM (
SELECT ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT') AS CR_DR_DEP_WITH ,SUM(TRAN_AMNT) AS TRAN_CREDIT   
 FROM  FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG  
WHERE ENTITY_TYPE = 'I' 
GROUP BY ENTITY_ID,ENTITY_TYPE,COALESCE(CR_DR_DEP_WITH,'CREDIT')
HAVING COALESCE(CR_DR_DEP_WITH,'CREDIT') = 'CREDIT'
)T
WHERE ENTITY_ID = PERSON_MASTER_ID'''


In [11]:
# config = ConfigParser()
# config.read(r'config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [12]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [13]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


In [14]:
#Run these queries to populate risk_person_summary_stg table




# Making a list of queries to populate the risk_person_summary_stg table
per_queries = [master_data_pop_per_q1,master_data_pop_per_q2,per_pincode_update,per_occ_update,master_data_pop_per_income,tran_amnt,peer_grouping_columns,MOBILE_COUNT,TELEPHONE_COUNT,EMAIL_COUNT,WATCHLIST_COUNT,STR_REPORT_COUNT,CTR_REPORT_COUNT,NTR_REPORT_COUNT,PTR_COUNT,CBWTR_REPORT_COUNT,CCR_REPORT_COUNT,str_amount_debit,str_amount_credit,ctr_amount_debit,ctr_amount_credit,ntr_amount_debit,ntr_amount_credit,cbwtr_amount_debit,cbwtr_amount_credit,ccr_amount_debit,ccr_amount_credit]
for query in per_queries:
    cursor.execute(query)
    print(namestr(query,globals())[0]," DONE!")
    conn.commit()

cursor.execute('''Delete from [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_Incremental] ''')
conn.commit()
cursor.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_Incremental] 
(
persON_MASTER_ID
)
SELECT persON_MASTER_ID
FROM [FINCORE_DB].[Fincore_Analytics].[risk_person_summary_stg] with (nolock)''')
conn.commit()
conn.close()

master_data_pop_per_q1  DONE!
master_data_pop_per_q2  DONE!
per_pincode_update  DONE!
per_occ_update  DONE!
master_data_pop_per_income  DONE!
per_trx_update_credit  DONE!
per_trx_update_debit  DONE!
peer_grouping_columns  DONE!
MOBILE_COUNT  DONE!
TELEPHONE_COUNT  DONE!
EMAIL_COUNT  DONE!
WATCHLIST_COUNT  DONE!
STR_REPORT_COUNT  DONE!
CTR_REPORT_COUNT  DONE!
NTR_REPORT_COUNT  DONE!
PTR_COUNT  DONE!
CBWTR_REPORT_COUNT  DONE!
CCR_REPORT_COUNT  DONE!
str_amount_debit  DONE!
str_amount_credit  DONE!
ctr_amount_debit  DONE!
ctr_amount_credit  DONE!
ntr_amount_debit  DONE!
ntr_amount_credit  DONE!
cbwtr_amount_debit  DONE!
cbwtr_amount_credit  DONE!
ccr_amount_debit  DONE!
ccr_amount_credit  DONE!


In [15]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [16]:
#Populate the risk_person_summary_stg_rules

for query in rules_df[(rules_df['IS_ACTIVE_FLAG'] == 1) & (rules_df['PERSON_TYPE'] == 'Individual')]['Query'].to_list():
    pprint(query)
    cursor.execute(query)
    print("Done")
#     print(namestr(query,globals())[0]," DONE!")
    conn.commit()
    
conn.close()

('MERGE INTO '
 'FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES          \r\n'
 'WITH (HOLDLOCK) as target USING           \r\n'
 '(         \r\n'
 'select Distinct k.Person_Master_Id,      \r\n'
 "'P001001000'  as rule_no     ,     \r\n"
 'w.Agency_Name as rule_value,           \r\n'
 '10 as  risk_score_static ,               \r\n'
 'GETDATE() as create_date                \r\n'
 'from FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL k with (nolock)\r\n'
 'inner join [FINCORE_DB].[Fincore].[Watchlist] W  with '
 '(nolock)              \r\n'
 'on k.PERSON_MASTER_ID = W.Entity_Master_Id             \r\n'
 "where w.Entity_Type = 'I'      and W.Is_Active_Flag = 1    ) \r\n"
 'AS source ON                          target.PERSON_MASTER_ID = '
 'source.Person_Master_id          \r\n'
 "and target.rule_no ='P001001000'              \r\n"
 'and target.rule_value  =  source.rule_value             \r\n'
 'WHEN MATCHED THEN       \r\n'
 'UPDATE SET target.RULE_VALUE=source.Rule_va

 'COUNT(DISTINCT PMAM.ACCOUNT_MASTER_ID) AS NO_OF_ACCOUNTS ,     \r\n'
 "COUNT(DISTINCT ( CAST(RE_ID AS VARCHAR)  +'|'+ BRANCH_CODE ) ) AS "
 'NO_OF_BANKS   \r\n'
 'FROM [FINCORE_DB].[FINCORE].PERSON_MASTER_ACCOUNT_MASTER PMAM   WITH '
 '(NOLOCK)  \r\n'
 'INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_INCREMENTAL K WITH '
 '(NOLOCK)   \r\n'
 'ON PMAM.PERSON_MASTER_ID  = K.PERSON_MASTER_ID   \r\n'
 'INNER JOIN [FINCORE_DB].[FINCORE].ACCOUNT_MASTER AM  WITH (NOLOCK)\r\n'
 'ON PMAM.ACCOUNT_MASTER_ID = AM.ACCOUNT_MASTER_ID              \r\n'
 "WHERE PMAM.IS_ACTIVE_FLAG = 1    AND AM.ACCOUNT_STATUS = 'ACTIVE'   \r\n"
 'GROUP BY PMAM.PERSON_MASTER_ID )     \r\n'
 'AS SOURCE ON TARGET.PERSON_MASTER_ID = SOURCE.PERSON_MASTER_ID      AND '
 "TARGET.RULE_NO ='P002001002'   \r\n"
 'AND TARGET.RULE_VALUE = SOURCE.NO_OF_BANKS \r\n'
 'WHEN MATCHED THEN           \r\n'
 'UPDATE SET TARGET.RULE_VALUE=SOURCE.NO_OF_BANKS \r\n'
 'WHEN NOT MATCHED BY TARGET THEN        \r\n'
 'INSERT (PERSON_MASTER_I

Done
('MERGE INTO [FINCORE_DB].[FINCORE_ANALYTICS].[RISK_PERSON_SUMMARY_STG_RULES] '
 'AS     \r\n'
 'TARGET USING (                      \r\n'
 'SELECT W.PERSON_MASTER_ID,      \r\n'
 'W.INCOME_PER_YEAR,            \r\n'
 'SUM(W.SUM_AMOUNT) AS SUM_CREDIT ,     \r\n'
 'CAST(((SUM(W.SUM_AMOUNT)-W.INCOME_PER_YEAR)/W.INCOME_PER_YEAR)*100 AS '
 'DECIMAL(16,2)) AS AMNT_GRTR_THN_INCOME     \r\n'
 'FROM           (         \r\n'
 'SELECT  PM.PERSON_MASTER_ID,      \r\n'
 'T.SUM_AMOUNT,            \r\n'
 "CASE WHEN CBDTIR.INCOME_RANGE='0' THEN (CBDTIR.MIN_VALUE_LAKH + "
 'CBDTIR.MAX_VALUE_LAKH)/2   \r\n'
 "WHEN CBDTIR.INCOME_RANGE='1' THEN (CBDTIR.MIN_VALUE_LAKH + "
 'CBDTIR.MAX_VALUE_LAKH)/2       \r\n'
 "WHEN CBDTIR.INCOME_RANGE='2' THEN (CBDTIR.MIN_VALUE_LAKH + "
 'CBDTIR.MAX_VALUE_LAKH)/2    \r\n'
 "WHEN CBDTIR.INCOME_RANGE='3' THEN (CBDTIR.MIN_VALUE_LAKH + "
 'CBDTIR.MAX_VALUE_LAKH)/2    \r\n'
 "WHEN CBDTIR.INCOME_RANGE='4' THEN (CBDTIR.MIN_VALUE_LAKH + "
 'CBDTIR.MAX_VALUE_LAKH)/2  \r\n'